In [5]:
try:
    import pandas as pd
    import os

    from glob import glob

except:
    !pip install pandas
    !pip install os
    !pip install glob

finally:
    import pandas as pd
    import os

    from glob import glob

In [6]:
#Path Values
path_data_test = './Data/test'
path_data_train = './Data/train'

path_label_test = './Data/sample_submission.csv'
path_label_train = './Data/train_labels.csv'

In [7]:
#This function creates a data frame which includes path, id and label of each file
def pathLabel(path_data, path_label):
    df = pd.DataFrame({'path':glob(os.path.join(path_data,'*.tif'))})
    df['id'] = df.path.map(lambda x: x.split('\\')[1].split(".")[0])
    labels = pd.read_csv(path_label)
    df = df.merge(labels, on = "id")
    return df

In [8]:
#Reads batch size of images using pathLabel()'s dataframe function
#Returns tensor object of images
try:
    import numpy as np
    import cv2

    from tqdm import tqdm_notebook,trange
    
except:
    !pip install numpy
    ##!pip install cv2 #if this do not work try open-cv instead of cv2
    !pip install tqdm
finally:
    import numpy as np
   ## import cv2

    from tqdm import tqdm_notebook,trange
    
def readImages( start_index = 0 ,batch_size = 100, data_frame = None):
    X = np.zeros([batch_size,96,96,3],dtype=np.uint8) 
    y = data_frame['label'][0:batch_size]
    for i, row in tqdm_notebook(data_frame.iterrows(), total=batch_size):
        if i == batch_size:
            break
        X[i] = cv2.imread(row['path'])          
    return X,y, (start_index+batch_size)

In [9]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchbearer import Trial

In [10]:
#IMPORTING USING PYTORCH
try:
    import torch
except:
    !pip install https://download.pytorch.org/whl/cu90/torch-1.0.1-cp36-cp36m-win_amd64.whl
    !pip install torchvision
try: 
    import torchbearer
except:
    !pip install torchbearer

In [11]:
#Data Path Constants
path_train_images = "./Data/train"
path_train_labels = "./Data/train_labels.csv"

In [12]:
#move Images Based On Their Labels
df_train = pathLabel(path_data_train,path_label_train)

import glob
import shutil
import os

src_dir = "./Data/train"
dst_dir_true = "./Data/train/True"
dst_dir_false = "./Data/train/False"

for (im_path,im_label) in zip(df_train['path'], df_train['label']):
    if(im_label == 1):
        shutil.move(im_path, dst_dir_true)
    elif(im_label == 0):
        shutil.move(im_path, dst_dir_false)
    

In [13]:
#read images in batches
def load_images(image_size, batch_size, root):

    transform = transforms.Compose([
                    transforms.Resize(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    train_set = torchvision.datasets.ImageFolder(root=root, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    return train_loader

In [14]:
#TEST LOADING IMAGES
train_loader = load_images(image_size=96, batch_size=128, root="./Data/train")
train_loader

In [ ]:
index = 0
for image in train_loader:
    index += 1
print(str(index)+" Index should be ="+str(220025/128))